In [6]:
import sys
import math
import numpy as np
"""
not working
from Language_Data_Scraper.py import *
"""
sys.path.insert(0, '..')
from net_framework import *
from sklearn.model_selection import train_test_split
import pandas as pd

In [7]:
term_data = pd.read_csv('term.txt', sep="\t", header=None)
term_data.columns = ["#Lnum", "#snum", "#cnum", "Term Abbrev"]
term_data.head()

,#Lnum,#snum,#cnum,Term Abbrev
0,1,1,1,LB
1,1,1,2,LB
2,1,1,3,LE
3,1,1,4,WK
4,1,1,5,LF


In [8]:
cnum_data = pd.read_csv('cnum-vhcm-lab-new.txt', sep="\t")
locations = cnum_data[['#cnum']]
locations['Normalized-L'] = (cnum_data['L*'] - cnum_data['L*'].mean())/(cnum_data['L*'] - cnum_data['L*'].mean()).max()
locations['Normalized-a'] = (cnum_data['a*'] - cnum_data['a*'].mean())/(cnum_data['a*'] - cnum_data['a*'].mean()).std() * 1/2
locations['Normalized-b'] = (cnum_data['b*'] - cnum_data['b*'].mean())/(cnum_data['b*'] - cnum_data['b*'].mean()).std() * 1/2
display(locations.head(5))

,#cnum,Normalized-L,Normalized-a,Normalized-b
0,141,1.000000,-0.025675,-0.138822
1,274,0.876301,-0.025504,-0.138822
2,129,0.876301,0.070445,-0.106039
3,230,0.876301,0.070101,-0.089951
4,302,0.876301,0.070617,-0.072041


In [9]:
locations = locations.sort_values('#cnum')
chip_num = list(locations['#cnum'])
lab_norm = [[row[2], row[3], row[4]] for row in locations.itertuples()]
print(chip_num)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

In [10]:
l1 = term_data[term_data.get('#Lnum').eq(1)]
unique_symbols = list(l1['Term Abbrev'].unique())
l1_grouped = l1.groupby('#cnum')['Term Abbrev'].apply(list)
l1_chip_abbrev_percentage = [[(l1_grouped[i + 1].count(abbrev) / len(l1_grouped[i + 1])) for abbrev in unique_symbols] for i in range(len(l1_grouped))]

In [11]:
l1_result = pd.DataFrame(l1_chip_abbrev_percentage)
l1_result.index += 1
l1_result.index.name = '#cnum'
l1_result.columns = unique_symbols
print(l1_result)
chip_norm = []
#pull the percentage for each cnum
for x in chip_num:
    chip_norm.append((l1_result.loc[l1["#cnum"]==x]).values.tolist()[0])

         LB    LE    WK    LF     F     G     S   GB    FU
#cnum                                                     
1      0.36  0.00  0.00  0.04  0.08  0.52  0.00  0.0  0.00
2      0.12  0.00  0.04  0.12  0.60  0.04  0.00  0.0  0.08
3      0.00  0.96  0.04  0.00  0.00  0.00  0.00  0.0  0.00
4      0.28  0.00  0.40  0.00  0.08  0.04  0.20  0.0  0.00
5      0.04  0.00  0.00  0.20  0.68  0.08  0.00  0.0  0.00
...     ...   ...   ...   ...   ...   ...   ...  ...   ...
326    0.08  0.20  0.36  0.00  0.20  0.00  0.12  0.0  0.04
327    0.04  0.00  0.00  0.68  0.20  0.04  0.04  0.0  0.00
328    0.64  0.00  0.08  0.00  0.00  0.20  0.08  0.0  0.00
329    0.08  0.08  0.40  0.00  0.12  0.00  0.28  0.0  0.04
330    0.04  0.00  0.00  0.72  0.24  0.00  0.00  0.0  0.00

[330 rows x 9 columns]


In [15]:
#Number of training iterations
num_iters = 5000
#Size of training dataset
num_examples = 2000
#Listing out the shapes of each model
colors_num = len(chip_norm[0])
input_size = 3

network_shapes = [(input_size, [100], colors_num), (input_size, [100], colors_num), (input_size, [100], colors_num)]
#Learning rate of the network
rate = 0.001

#Generating Training Data
lab_train, lab_test, chip_train, chip_test = train_test_split(lab_norm, chip_norm, test_size=0.1)

input_train = torch.FloatTensor(lab_train)
output_train = torch.FloatTensor(chip_train)
input_test= torch.FloatTensor(lab_test)
output_test = torch.FloatTensor(chip_test)

print(chip_train)

[[0.0, 0.44, 0.28, 0.0, 0.16, 0.0, 0.12, 0.0, 0.0], [0.0, 0.28, 0.36, 0.04, 0.28, 0.0, 0.04, 0.0, 0.0], [0.0, 0.0, 0.0, 0.56, 0.4, 0.0, 0.0, 0.0, 0.04], [0.68, 0.0, 0.0, 0.0, 0.0, 0.28, 0.04, 0.0, 0.0], [0.0, 0.0, 0.0, 0.64, 0.28, 0.0, 0.04, 0.0, 0.04], [0.44, 0.0, 0.0, 0.04, 0.0, 0.52, 0.0, 0.0, 0.0], [0.32, 0.0, 0.0, 0.0, 0.04, 0.6, 0.04, 0.0, 0.0], [0.64, 0.0, 0.0, 0.0, 0.0, 0.32, 0.04, 0.0, 0.0], [0.12, 0.0, 0.04, 0.04, 0.36, 0.4, 0.04, 0.0, 0.0], [0.12, 0.0, 0.0, 0.08, 0.36, 0.36, 0.04, 0.0, 0.04], [0.56, 0.0, 0.0, 0.0, 0.08, 0.28, 0.08, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.12, 0.0, 0.08, 0.12, 0.24, 0.36, 0.04, 0.0, 0.04], [0.0, 0.4, 0.4, 0.0, 0.12, 0.0, 0.08, 0.0, 0.0], [0.96, 0.0, 0.0, 0.04, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.08, 0.52, 0.4, 0.0, 0.0, 0.0, 0.0], [0.04, 0.0, 0.0, 0.2, 0.68, 0.08, 0.0, 0.0, 0.0], [0.36, 0.04, 0.2, 0.0, 0.28, 0.0, 0.12, 0.0, 0.0], [0.0, 0.4, 0.28, 0.0, 0.12, 0.0, 0.2, 0.0, 0.0], [0.12, 0.08, 0.12, 0.0, 0.12, 0.44, 0.08, 

In [16]:
#Array of losses over training period for each network
for net_num, shape in enumerate(network_shapes):
    print('Network Shape:',flush = True)
    print('Input Size = ' + str(shape[0]), flush = True)
    print('Hidden Size = ' + str(shape[1]), flush = True)
    print('Output Size = ' + str(shape[2]), flush = True)
    NN = Neural_Network(inputSize = 3, outputSize = colors_num, hiddenSize = [3,3,3] , learning_rate = 0.001)

    for i in range(num_iters):
        #Calculating l1 error
        error = NN.l1error(output_train, NN(input_train))
        if i == 0: 
            dh = display("#" + str(i) + " Error: " + str(error), display_id=True)
        else:
            dh.update("#" + str(i) + " Error: " + str(error))
            
        NN.train(input_train, output_train)
    #Saves the training results in a filed named "Net 0", "Net 1", etc. 
    NN.saveWeights(model = NN, path = "saved_networks/Net " + str(net_num));

Network Shape:
Input Size = 3
Hidden Size = [100]
Output Size = 9


'#0 Error: 0.3937310576438904'

Network Shape:
Input Size = 3
Hidden Size = [100]
Output Size = 9


'#0 Error: 0.4112871289253235'

Network Shape:
Input Size = 3
Hidden Size = [100]
Output Size = 9


'#0 Error: 0.3904036581516266'

In [17]:
for net_num, shape in enumerate(network_shapes):
    print('Network Shape:',flush = True)
    print('Input Size = ' + str(shape[0]), flush = True)
    print('Hidden Size = ' + str(shape[1]), flush = True)
    print('Output Size = ' + str(shape[2]), flush = True)
    # Loading the network we trained in the prev. section
    
    NN = Neural_Network(inputSize = shape[0], outputSize = shape[2],
                        hiddenSize = shape[1] , learning_rate = rate)
    NN.load_state_dict(torch.load("saved_networks/Net " + str(net_num)))
    
    validation_error = NN.l1error(output_test, NN(input_test))
    max_error = np.max(np.abs((output_test - NN(input_test)).detach().numpy()))
    print("The validation error is: " + str(validation_error), flush = True)  
    print("The maximum error is:" + str(max_error), flush = True)

Network Shape:
Input Size = 3
Hidden Size = [100]
Output Size = 9


RuntimeError: Error(s) in loading state_dict for Neural_Network:
	Unexpected key(s) in state_dict: "model.4.weight", "model.4.bias", "model.6.weight", "model.6.bias". 
	size mismatch for model.0.weight: copying a param with shape torch.Size([3, 3]) from checkpoint, the shape in current model is torch.Size([100, 3]).
	size mismatch for model.0.bias: copying a param with shape torch.Size([3]) from checkpoint, the shape in current model is torch.Size([100]).
	size mismatch for model.2.weight: copying a param with shape torch.Size([3, 3]) from checkpoint, the shape in current model is torch.Size([9, 100]).
	size mismatch for model.2.bias: copying a param with shape torch.Size([3]) from checkpoint, the shape in current model is torch.Size([9]).